# Fractional Brownian motion application

## CPU info

In [1]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 25
model		: 17
model name	: AMD EPYC 9B14
stepping	: 1
microcode	: 0xffffffff
cpu MHz		: 2599.998
cache size	: 1024 KB
physical id	: 0
siblings	: 8
core id		: 0
cpu cores	: 4
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 16
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext invpcid_single ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid avx512f avx512dq rdseed adx smap avx512ifma clflushopt clwb avx512cd sha_ni avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves avx512_bf16 clzero xsaveerptr wbnoinvd arat avx512v

Install relevant packages which are not installed yet

In [3]:
!pip install PyGithub
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 17.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 48.4 MB/s eta 0:00:0000:0100:01


## Run for $\mathcal{N}_T=9$

### Final run (for H\in{0.2,0.3, 0.45, ,0.7\})

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
# auth = Auth.Token('**********')- Outcommented + deleted due to security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 700000
traj_est_MM = 8000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=15000
traj_lb_testing = 100000
information=[]
steps=9


T=1
basis_f_K_U_MM=80
basis_f_K_U_MM_desai2=50
basis_f_K_U_MM_belomestny2013=230
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.15)

K_L_basic= 250
K_U_belomestny2009= 200
K_U_SZH2013=160
K_L_AndersenBroadie2004=250
K_L_Glasserman2004= 250
grid_Glasserman2004= 2400
inner_sim_SZH_=1
inner_sim_AB= 1000 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//2
traj_lb_testing_GM=traj_test_ub//2


i=0
if __name__=='__main__':
    for d,H in  [(1,0.2), (1,0.3), (1, 0.45), (1, 0.7)]:
        for inner_sim in [600]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)


********** 600 **********
---------- Schoenmakers et al. (2013) KU100 -9 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
Lowerbound
Value 0.3610490448286773
Std 0.0026944690644603547
Stop time-mean 5.11545
Stop time-std 2.5928751025647183

Upperbound
up 0.38474580803729636
std 0.0012841681882568075
up2 0.3746572508372763
std2 0.0014323533365769312
CV est 0.3612960002906745
CV std 0.0014005187189053791
time avg training 0:02:14.326467
time avg ub 0:00:00.059664
---------- Belom. et al. (2009) KU100-9 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
0
1
2
3
4
5
6
7
8
[0.00813707 0.00940951 0.00771455]
Lowerbound
Value 0.36012947776892434
Std 0.002701927171374924
Stop time-mean 5.14288
Stop time-std 2.5826547011941026
Upperbound
up 0.38489045564328617
std 0.001298647961929795
up2 0.37412934163220646
std2 0.0014488105404998706
CV est 0.3605423335944729
CV std 0.0014074798064049251
time avg training 0:01:31.036513
time avg ub 0:00:00.044396
---------- Glasserman (2004)-9 2400 1 ----------


...

8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.19111352375401525   - Norm Jacobian 2.946796726274097 - Max abs Jacobian 0.11680362401827572
Iteration: 10, Objective Value: 0.16957096816583972   - Norm Jacobian 1.022475048824272 - Max abs Jacobian 0.04519961553271263
Iteration: 20, Objective Value: 0.16556738848421304   - Norm Jacobian 0.17943280868890465 - Max abs Jacobian 0.012334073672472412
Iteration: 30, Objective Value: 0.16360896642026607   - Norm Jacobian 0.12696663638436337 - Max abs Jacobian 0.008540707579184894
Iteration: 40, Objective Value: 0.16266461678284003   - Norm Jacobian 0.07214089358148161 - Max abs Jacobian 0.004581321148398969
Iteration: 50, Objective Value: 0.16239989239193728   - Norm Jacobian 0.01797491046860102 - Max abs Jacobian 0.0016302621710826658
Iteration: 60, Objective Value: 0.1623463242622934   - Norm Jacobian 0.006800497577473615 - Max abs Jacobian 0.0007432623136780872
Iteration: 70, Objective Value: 0.1623378574495354   - Norm Jacobian 0.004263

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.2493426320276623   - Norm Jacobian 6.37283628493215 - Max abs Jacobian 0.23129630280622274
Iteration: 10, Objective Value: 0.1923655310005436   - Norm Jacobian 4.789839235334749 - Max abs Jacobian 0.2384879181331636
Iteration: 20, Objective Value: 0.18223599794598155   - Norm Jacobian 0.6972551249261204 - Max abs Jacobian 0.04922276572943523
Iteration: 30, Objective Value: 0.17954328173910697   - Norm Jacobian 0.28449851821470806 - Max abs Jacobian 0.01630992518709791
Iteration: 40, Objective Value: 0.17891053340726654   - Norm Jacobian 0.17749173698999096 - Max abs Jacobian 0.010595617858054431
Iteration: 50, Objective Value: 0.1786492180332293   - Norm Jacobian 0.04366298126170164 - Max abs Jacobian 0.0038626256743736656
Iteration: 60, Objective Value: 0.1785550430536636   - Norm Jacobian 0.02818085867988624 - Max abs Jacobian 0.0016716738093566776
Iteration: 70, Objective Value: 0.1785268927340315   - Norm Jacobian 0.013153104037120

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.31004743796534195   - Norm Jacobian 10.542927644745951 - Max abs Jacobian 0.5085864763288614
Iteration: 10, Objective Value: 0.2220707423109242   - Norm Jacobian 6.7074449790919335 - Max abs Jacobian 0.31972211434207076
Iteration: 20, Objective Value: 0.20109388910064271   - Norm Jacobian 3.007857063602664 - Max abs Jacobian 0.17528131546664208
Iteration: 30, Objective Value: 0.19524827996393707   - Norm Jacobian 0.6029282878626456 - Max abs Jacobian 0.04742714495255727
Iteration: 40, Objective Value: 0.19370018040904183   - Norm Jacobian 0.2597070977652238 - Max abs Jacobian 0.018153968188927665
Iteration: 50, Objective Value: 0.19319996054096156   - Norm Jacobian 0.13019876388716278 - Max abs Jacobian 0.011521682639963223
Iteration: 60, Objective Value: 0.19300525155923465   - Norm Jacobian 0.10491489784955853 - Max abs Jacobian 0.006902009574859696
Iteration: 70, Objective Value: 0.19292074418150754   - Norm Jacobian 0.0570555799094

## Final Run $\mathcal{N}_T=49$ (24-6-2024) 

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
# auth = Auth.Token('**********')- Outcommented + deleted due to security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 700000
traj_est_MM = 2000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=6500
traj_lb_testing = 100000
information=[]
steps=49


T=1
basis_f_K_U_MM=20
basis_f_K_U_MM_desai2=15
basis_f_K_U_MM_belomestny2013=65
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.35)

K_L_basic=200
K_U_belomestny2009= 100
K_U_SZH2013=75
K_L_AndersenBroadie2004=115
K_L_Glasserman2004= 225
grid_Glasserman2004= 1850
inner_sim_SZH_=1
inner_sim_AB= 620 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//7
traj_lb_testing_GM=traj_test_ub//4


i=0
if __name__=='__main__':
    for d,H in  [(1,0.2), (1,0.45)]:
        for inner_sim in [500]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004 if H!=0.45 else int(1.2*K_L_AndersenBroadie2004)
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            inner_sim_AB_=inner_sim_AB if H!=0.45 else int(1.2*inner_sim_AB)
            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB_, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB_, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)



********** 500 **********
---------- Schoenmakers et al. (2013) KU100 -49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Lowerbound
Value 0.5859255660388076
Std 0.0025825279435969114
Stop time-mean 25.43839
Stop time-std 13.706915926199446

Upperbound
up 0.6373973294320792
std 0.0013285695542543463
up2 0.6206365803387507
std2 0.001594869477626332
CV est 0.5857335054302835
CV std 0.0014919576788036553
time avg training 0:08:13.791120
time avg ub 0:00:00.198085
---------- Belom. et al. (2009) KU100-49 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


t= 48
t= 47
t= 46
t= 45
t= 44
t= 43
t= 42
t= 41
t= 40
t= 39
t= 38
t= 37
t= 36
t= 35
t= 34
t= 33
t= 32
t= 31
t= 30
t= 29
t= 28
t= 27
t= 26
t= 25
t= 24
t= 23
t= 22
t= 21
t= 20
t= 19
t= 18
t= 17
t= 16
t= 15
t= 14
t= 13
t= 12
t= 11
t= 10
t= 9
t= 8
t= 7
t= 6
t= 5
t= 4
t= 3
t= 2
t= 1
t= 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
[-0.00460477 -0.005716   -0.00190481]
Lowerbound
Value 0.5850778762878759
Std 0.002582906654153012
Stop time-mean 25.44513
Stop time-std 13.708763229522201
Upperbound
up 0.634675419704042
std 0.001313974521739566
up2 0.6182983448938371
std2 0.0015693768922266824
CV est 0.5853595775505334
CV std 0.001475555929076679
time avg training 0:05:36.165276
time avg ub 0:00:00.150084
---------- Glasserman (2004)-49 1850 1 ----------


...

48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.24727619771078688   - Norm Jacobian 4.850604886255864 - Max abs Jacobian 0.15942071682903336
Iteration: 10, Objective Value: 0.11622681154553358   - Norm Jacobian 2.4145405779932565 - Max abs Jacobian 0.16001715337702277
Iteration: 20, Objective Value: 0.09336059762527725   - Norm Jacobian 1.6239484676840767 - Max abs Jacobian 0.0779123992024902
Iteration: 30, Objective Value: 0.0873191009416969   - Norm Jacobian 1.0537869005309142 - Max abs Jacobian 0.052971953788695354
Iteration: 40, Objective Value: 0.08462020502101883   - Norm Jacobian 0.5759843290568333 - Max abs Jacobian 0.03737198917544012
Iteration: 50, Objective Value: 0.08339734912464977   - Norm Jacobian 0.44968301598739047 - Max abs Jacobian 0.02981855448172784
Iteration: 60, Objective Value: 0.08258621783233491   - Norm Jacobian 0.41288546699447226 - Max a

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.35621462098449197   - Norm Jacobian 6.656005740088506 - Max abs Jacobian 0.29495679436616873
Iteration: 10, Objective Value: 0.18072765280792188   - Norm Jacobian 3.676440314497514 - Max abs Jacobian 0.1686046595096042
Iteration: 20, Objective Value: 0.1340984065639113   - Norm Jacobian 4.054589818442622 - Max abs Jacobian 0.2366064548527852
Iteration: 30, Objective Value: 0.1143363590921129   - Norm Jacobian 2.7152077570155684 - Max abs Jacobian 0.10808285711031529
Iteration: 40, Objective Value: 0.10516391872718318   - Norm Jacobian 1.7496198151648812 - Max abs Jacobian 0.09021332546422121
Iteration: 50, Objective Value: 0.10175441163020497   - Norm Jacobian 1.3203931438714256 - Max abs Jacobian 0.07214921116172036
Iteration: 60, Objective Value: 0.09984103926345338   - Norm Jacobian 0.9838455883787716 - Max abs Jaco

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


48
47
46
45
44
43
42
41
40
39
38
37
36
35
34
33
32
31
30
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.4598623831305267   - Norm Jacobian 8.822615263229842 - Max abs Jacobian 0.41763310002872
Iteration: 10, Objective Value: 0.25989381268122597   - Norm Jacobian 5.011465845394628 - Max abs Jacobian 0.2837080269460883
Iteration: 20, Objective Value: 0.1579279377267591   - Norm Jacobian 4.119344486800211 - Max abs Jacobian 0.2838048156424613
Iteration: 30, Objective Value: 0.13790564343240727   - Norm Jacobian 3.416380756585304 - Max abs Jacobian 0.18045536159204362
Iteration: 40, Objective Value: 0.1264747429266479   - Norm Jacobian 2.401441316833915 - Max abs Jacobian 0.13697801571838097
Iteration: 50, Objective Value: 0.12093503028382609   - Norm Jacobian 1.9791195819732783 - Max abs Jacobian 0.08861898950828451
Iteration: 60, Objective Value: 0.11789129750617997   - Norm Jacobian 1.7078726782794629 - Max abs Jacobian 0

## $\mathcal{N}_T=9$ with higher number of inner simulations AB, hurst close to 0.5

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow import keras
import os
import sys
# parent_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))
# sys.path.append(parent_dir)
import utils
import github
from github import Auth
from github import Github
# auth = Auth.Token('**********')- Outcommented + deleted due to security reasons
g = github.Github(auth=auth)
repo = g.get_repo('PatrickvanEwijk/UpperboundApproachesDuality')

from fBMABLS import main as AndersonBroadie2004
from fBMHK import main as Glassermann
from fBMBelomestny import main as mainBelomestnyetal2009
from fBMSAA import main as SAA_LP
from fBMSAABelomestny import main as BelomestnySAA2013
from fBMSchoenmakersSZH import main as SZH2013

import numpy as np
import pandas as pd
np.set_printoptions(edgeitems=20, linewidth=200)
from tabulate import tabulate
from datetime import datetime
from itertools import product
import pickle as pic


traj_test_ub= 10000 
traj_est_primal_dual = 700000
traj_est_MM = 8000
traj_test_ub_desai2=30000
traj_est_MM_belomestny2013=15000
traj_lb_testing = 100000
information=[]
steps=9


T=1
basis_f_K_U_MM=80
basis_f_K_U_MM_desai2=50
basis_f_K_U_MM_belomestny2013=230
basis_f_K_U_MM_BHS=int(basis_f_K_U_MM*1.15)

K_L_basic= 250
K_U_belomestny2009= 200
K_U_SZH2013=160
K_L_AndersenBroadie2004=100
K_L_Glasserman2004= 250
grid_Glasserman2004= 2400
inner_sim_SZH_=1
inner_sim_AB= 2400 
fine_grid_belomestny2009_= 1

traj_test_ub_AB=traj_test_ub//2
traj_lb_testing_GM=traj_test_ub//2


i=0
if __name__=='__main__':
    for d,H in  [(1,0.4), (1, 0.45), (1, 0.5), (1, 0.55), (1, 0.6)]:
        for inner_sim in [600]:
            print(''.join(np.repeat('*', 10)), inner_sim ,''.join(np.repeat('*', 10)))     
            # calibrations_= [[keras.activations.softplus, 30, 'HeNormal', 1]]
            # for activation_f, vs_factor, distribution_vs, w_c in calibrations_:
            inf_cols = [d, H, '', '', '', '']


            label_=f'Schoenmakers et al. (2013) KU100 -{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim_SZH_, d,  ''.join(np.repeat('-', 10)))
            list_inf=SZH2013(d, True, hurst=H, steps=steps,T=T, mode_kaggle=True, grid=1, K_low=K_L_basic,K_up=K_U_SZH2013, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_SZH_, info_cols=inf_cols)
            information.append(inf_list)    

            label_=f'Belom. et al. (2009) KU100-{steps}'
            print(''.join(np.repeat('-', 10)),label_, fine_grid_belomestny2009_, d,  ''.join(np.repeat('-', 10)))
            list_inf=mainBelomestnyetal2009(d, True,hurst=H, steps=steps,T=T, mode_kaggle=True, grid=fine_grid_belomestny2009_, K_low=K_L_basic,K_up=K_U_belomestny2009, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= fine_grid_belomestny2009_, info_cols=inf_cols)
            information.append(inf_list)   


            K_L_AB=K_L_AndersenBroadie2004
            K_L_GM=K_L_Glasserman2004
            label_ = f'Glasserman (2004)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, grid_Glasserman2004, d, ''.join(np.repeat('-', 10)))
            list_inf=Glassermann(d, True, hurst=H, steps=steps,T=T, grid=grid_Glasserman2004, K_low=K_L_GM, traj_est=traj_est_primal_dual, traj_test_ub=traj_lb_testing_GM, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= grid_Glasserman2004, info_cols=inf_cols)
            information.append(inf_list)   

            label_ = f'Anderson Broadie (2004)-{steps}'
            print(''.join(np.repeat('-', 10)), label_, inner_sim_AB, d, ''.join(np.repeat('-', 10)))
            list_inf=AndersonBroadie2004(d, True, hurst=H, steps=steps,T=T, grid= inner_sim_AB, K_low=K_L_AB, traj_est=traj_est_primal_dual, traj_test_ub=traj_test_ub_AB, traj_test_lb=traj_lb_testing, seed=i+9) 
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim_AB, info_cols=inf_cols)
            information.append(inf_list)


            label_ = f'Belomestny et al. (2019)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_BHS,  traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9,  mode_desai_BBS_BHS='bhs')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) best-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs',  mean_BBS=1.0, std_BBS=0.0)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ = f'Belomestny et al. (2023) original-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps, T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='bbs', mean_BBS=0.6, std_BBS=0.3)
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012)-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d,  True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM, traj_est=traj_est_primal_dual, traj_train_ub=traj_est_MM,traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            label_ =f'Desai et al. (2012) high test-{steps}'
            print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))                             
            list_inf=SAA_LP(d, True, hurst=H, mode_kaggle=True, steps=steps,T=T,  grid=inner_sim, K_low=K_L_basic,K_up=basis_f_K_U_MM_desai2, traj_est=traj_est_primal_dual, traj_train_ub=int(traj_est_MM*1.5),traj_test_ub=traj_test_ub_desai2, traj_test_lb=traj_lb_testing, seed=i+9, mode_desai_BBS_BHS='desai')
            inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
            information.append(inf_list)  

            for lambda_ in [1/20, 1/2, 1]:
                label_ = f'Belomesty (2013); lambda={lambda_}'
                print(''.join(np.repeat('-', 10)),label_, inner_sim, d, ''.join(np.repeat('-', 10)))
                if lambda_==1/20:
                    traj_est_MM_belomestny2013_=int(1.1*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.1*basis_f_K_U_MM_belomestny2013)
                elif lambda_==1/2:
                    traj_est_MM_belomestny2013_=int(1.05*traj_est_MM_belomestny2013)
                    basis_f_K_U_MM_belomestny2013_=int(1.05*basis_f_K_U_MM_belomestny2013)
                else:
                    traj_est_MM_belomestny2013_=traj_est_MM_belomestny2013
                    basis_f_K_U_MM_belomestny2013_=basis_f_K_U_MM_belomestny2013

                list_inf=BelomestnySAA2013(d, True, hurst=H, steps=steps,T=T, grid=inner_sim, K_low=K_L_basic, K_up=basis_f_K_U_MM_belomestny2013_, traj_est=traj_est_primal_dual, traj_est_ub=traj_est_MM_belomestny2013_, traj_test_ub=traj_test_ub, traj_test_lb=traj_lb_testing, seed=i+9, lambda_=lambda_, p=100)
                inf_list=utils.process_function_output(*list_inf, label_ = label_, grid= inner_sim, info_cols=inf_cols)
                information.append(inf_list) 





            with open(f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.pic', 'wb') as fh:
                pic.dump(information, fh)
            repo.create_file(f'resultfiles/run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', f'run{datetime.now().strftime("%Y%m%d%H%m%S")}.txt', str(information), branch='main')

    info_original=pd.DataFrame.from_records(information)
    info_transformed=pd.DataFrame.from_records(utils.information_format_fbm_no_fu(information), columns=utils.header_fbm_no_fu)
    lower_bounds_cv=info_original.iloc[:,[2,13, 14]]
    upper_bounds = info_original.iloc[:, 9]
    loc_lowest_standard_error = lower_bounds_cv.groupby(2)[14].idxmin()
    lower_bounds_cv_sel = lower_bounds_cv.loc[loc_lowest_standard_error, [2, 13]].rename(columns={13: 'best'})
    lower_bounds_cv=pd.merge(left= lower_bounds_cv, right=lower_bounds_cv_sel, on=2)['best']
    lower_bounds_cv.index= info_original.index
    info_transformed['GAP']= upper_bounds- lower_bounds_cv
    table_ = tabulate(utils.information_format_fbm_no_fu(information), headers=utils.header_fbm_no_fu, tablefmt="latex_raw", floatfmt=".4f")

    print(table_)


2024-06-22 18:45:48.565703: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-22 18:45:49.320734: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 18:45:49.320809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 18:45:49.513262: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-22 18:45:49.736515: I tensorflow/core/platform/cpu_feature_guar

********** 600 **********
---------- Schoenmakers et al. (2013) KU100 -9 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
0
1
2
3
4
5
6
7
8
Lowerbound
Value 0.10062845373613265
Std 0.002473483029568516
Stop time-mean 5.06964
Stop time-std 2.7510234950650636

Upperbound
up 0.1063746790940483
std 0.0003512445071167226
up2 0.10418332195940393
std2 0.0003829199419614142
CV est 0.1008976087773144
CV std 0.00037534731467745266
time avg training 0:02:11.537135
time avg ub 0:00:00.055616
---------- Belom. et al. (2009) KU100-9 1 1 ----------


/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


...

8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.11978775126461037   - Norm Jacobian 4.304369541704544 - Max abs Jacobian 0.14644984154110485
Iteration: 10, Objective Value: 0.09733025493856146   - Norm Jacobian 1.8712518499291997 - Max abs Jacobian 0.07273808944195287
Iteration: 20, Objective Value: 0.09310167058152205   - Norm Jacobian 0.3447621003430763 - Max abs Jacobian 0.021532044233750477
Iteration: 30, Objective Value: 0.09176668004451025   - Norm Jacobian 0.20341227047113042 - Max abs Jacobian 0.01636613424702874
Iteration: 40, Objective Value: 0.09076086896526323   - Norm Jacobian 0.0647921159547579 - Max abs Jacobian 0.004242253796579745
Iteration: 50, Objective Value: 0.0904372582054002   - Norm Jacobian 0.03780323878436859 - Max abs Jacobian 0.002865816891987162
Iteration: 60, Objective Value: 0.09034206072395162   - Norm Jacobian 0.024145547655632624 - Max abs Jacobian 0.001850176772935173
Iteration: 70, Objective Value: 0.09031122280613525   - Norm Jacobian 0.010067714

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.1655445659608524   - Norm Jacobian 8.203036870770095 - Max abs Jacobian 0.28986581546046325
Iteration: 10, Objective Value: 0.11524902213405272   - Norm Jacobian 3.677044550045078 - Max abs Jacobian 0.19839035332858992
Iteration: 20, Objective Value: 0.10774253491436066   - Norm Jacobian 1.512700853594811 - Max abs Jacobian 0.07067469588413461
Iteration: 30, Objective Value: 0.10568631680760766   - Norm Jacobian 0.3203371795552852 - Max abs Jacobian 0.0196187933002907
Iteration: 40, Objective Value: 0.10519857421514874   - Norm Jacobian 0.12475019316760567 - Max abs Jacobian 0.010135242021668859
Iteration: 50, Objective Value: 0.10497895584007143   - Norm Jacobian 0.06486152963323373 - Max abs Jacobian 0.005272487253879593
Iteration: 60, Objective Value: 0.10488963519305952   - Norm Jacobian 0.038393957269726275 - Max abs Jacobian 0.002279410089301062
Iteration: 70, Objective Value: 0.10485559571373325   - Norm Jacobian 0.0163281451844

/opt/conda/lib/python3.10/site-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


8
7
6
5
4
3
2
1
0
Iteration: 0, Objective Value: 0.2225892638722035   - Norm Jacobian 12.86562394887614 - Max abs Jacobian 0.6418833111085201
Iteration: 10, Objective Value: 0.14901675698726233   - Norm Jacobian 7.2447309107779185 - Max abs Jacobian 0.33663645862382696
Iteration: 20, Objective Value: 0.12703619652492598   - Norm Jacobian 4.018418246606412 - Max abs Jacobian 0.2308576680601646
Iteration: 30, Objective Value: 0.12061337560271371   - Norm Jacobian 0.9707278422932489 - Max abs Jacobian 0.05845182994161137
Iteration: 40, Objective Value: 0.11957278483366943   - Norm Jacobian 0.320342694277699 - Max abs Jacobian 0.016233818609992318
Iteration: 50, Objective Value: 0.11916489985589108   - Norm Jacobian 0.1289616741705011 - Max abs Jacobian 0.00826973415847961
Iteration: 60, Objective Value: 0.11897852137467863   - Norm Jacobian 0.07738310453086483 - Max abs Jacobian 0.004806946444079993
Iteration: 70, Objective Value: 0.11889465410706054   - Norm Jacobian 0.05290878690862503 